In [1]:
%pip install -qU langchain-community pymupdf


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/Users/jorge-jrzz/Desktop/PT/RAG-api-rest/dev/example.pdf")

In [27]:
docs = loader.load()
docs[0]

Document(metadata={'source': '/Users/jorge-jrzz/Desktop/PT/RAG-api-rest/dev/example.pdf', 'file_path': '/Users/jorge-jrzz/Desktop/PT/RAG-api-rest/dev/example.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.7', 'title': '', 'author': 'Jorge Angel Juarez Vazquez', 'subject': '', 'keywords': '', 'creator': 'OCRmyPDF 16.5.0 / Tesseract OCR-hOCR 5.4.1', 'producer': 'pikepdf 9.2.0', 'creationDate': "D:20241008160621+00'00'", 'modDate': "D:20241008162240+00'00'", 'trapped': ''}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta de opciones.\n¢ Desea insertar una imagen de sus archivos o agregar una forma o un cuadro de texto?\n¡Adelante! En la pestaña Insertar de la cinta de opciones, pulse la opción que necesite.\n')

In [28]:
docs[0].metadata = {"title": "Example PDF", "author": "John Doe"}
new_metadata = docs[1].metadata

In [29]:
new_metadata

{'source': '/Users/jorge-jrzz/Desktop/PT/RAG-api-rest/dev/example.pdf',
 'file_path': '/Users/jorge-jrzz/Desktop/PT/RAG-api-rest/dev/example.pdf',
 'page': 1,
 'total_pages': 2,
 'format': 'PDF 1.7',
 'title': '',
 'author': 'Jorge Angel Juarez Vazquez',
 'subject': '',
 'keywords': '',
 'creator': 'OCRmyPDF 16.5.0 / Tesseract OCR-hOCR 5.4.1',
 'producer': 'pikepdf 9.2.0',
 'creationDate': "D:20241008160621+00'00'",
 'modDate': "D:20241008162240+00'00'",
 'trapped': ''}

In [31]:
from pathlib import Path

mm = {
    'source': Path(docs[1].metadata['source']).name,
    'page': docs[1].metadata['page'],
    'author': docs[1].metadata['author'],
    'namespace': 'Espacio definido'
}
mm

{'source': 'example.pdf',
 'page': 1,
 'author': 'Jorge Angel Juarez Vazquez',
 'namespace': 'Espacio definido'}

In [ ]:
file_content = []
for doc in loader.lazy_load():
    doc.metadata = {
        'source': doc.metadata['source'],
        'page': doc.metadata['page'],
        'author': doc.metadata['author'],
        'namespace': 'Espacio definido'
    }
    file_content.append(doc)


file_content

[Document(metadata={'source': './example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'Espacio definido'}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta de opciones.\n¢ Desea insertar una imagen de sus archivos o agregar una forma o un cuadro de texto?\n¡Adelante! En la pestaña Insertar de la cinta de opciones, pulse la opción que necesite.\n'),
 Document(metadata={'source': './example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'Espacio definido'}, page_content='Esto es texto que esta en una imagen, en la segunda\npagina del documento, a continuación, información sobre\nlos perros:\nEl perro, llamado perro doméstico o can, y en algunos\nlugares coloquialmente llamado chucho, tu

In [32]:
"""
This module provides functionalities for Optical Character Recognition (OCR).
"""

import subprocess
from pathlib import Path
from typing import List
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document


class LoadFile:
    @staticmethod
    def _ocr_pdf(input_pdf: str, output_pdf: str , language='eng+spa') -> None:
        """
        Adds an OCR text layer to scanned PDF files, allowing them to be searched using OCRmyPDF.

        Args: 
            input_pdf (str): The path to the input PDF file.
            output_pdf (str): The path to the output PDF file.
            language (str): The language(s) to use for OCR. Default is 'eng+spa' (English and Spanish).
        
        Returns:
            None
        """
        try:
            # Construir el comando
            comando = [
                'ocrmypdf',
                '-l', language,
                '--force-ocr',
                '--jobs', '6',  # Número de trabajos en paralelo
                '--output-type', 'pdf',
                input_pdf,
                output_pdf
            ]

            # Ejecutar el comando
            subprocess.run(comando, check=True)
            print(f"OCR aplicado exitosamente a {input_pdf}. Salida: {output_pdf}")
        except subprocess.CalledProcessError as e:
            print(f"Error al aplicar OCR: {e}")

    @classmethod
    def load_file(cls, file_path: str, namespace: str) -> List[Document]:
        loader = PyMuPDFLoader(file_path)
        source = Path(file_path).name
        file_content = []
        for doc in loader.lazy_load():
            doc.metadata = {
                'source': source,
                'page': doc.metadata['page'],
                'author': doc.metadata['author'],
                'namespace': namespace
            }
            file_content.append(doc)

        return file_content

In [33]:
file = LoadFile().load_file('./example.pdf', 'jorgeang33@gmail.com')


In [34]:
file

[Document(metadata={'source': 'example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta de opciones.\n¢ Desea insertar una imagen de sus archivos o agregar una forma o un cuadro de texto?\n¡Adelante! En la pestaña Insertar de la cinta de opciones, pulse la opción que necesite.\n'),
 Document(metadata={'source': 'example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}, page_content='Esto es texto que esta en una imagen, en la segunda\npagina del documento, a continuación, información sobre\nlos perros:\nEl perro, llamado perro doméstico o can, y en algunos\nlugares coloquialmente llamado chucho